# Analyzing borrowers’ risk of defaulting

The goal of this project is to prepare a report for a bank’s loan division. Find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Report will be considered for building a **credit scoring** of a potential customer, which is used to evaluate the ability of a potential borrower to repay their loan.

Main steps:
- Preprocess the data
- Categorize the data

- Answer these questions:
-- Is there a connection between having kids and repaying a loan on time?
-- Is there a connection between marital status and repaying a loan on time?
-- Is there a connection between income level and repaying a loan on time?
-- How do different loan purposes affect on-time loan repayment?
- Analyze the results

# Description of the data

`children`: the number of children in the family

`days_employed`: how long the customer has been working

`dob_years`: the customer’s age

`education`: the customer’s education level

`education_id`: identifier for the customer’s education

`family_status`: the customer’s marital status

`family_status_id`: identifier for the customer’s marital status

`gender`: the customer’s gender

`income_type`: the customer’s income type

`debt`: whether the customer has ever defaulted on a loan

`total_income`: monthly income

`purpose`: reason for taking out a loan

## Open the data file and have a look at the general information. 

In [18]:
import pandas as pd

In [44]:
from pymystem3 import Mystem
from collections import Counter

In [19]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/credit_scoring_eng.csv')

df.info()
df.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,secondary education,1,married,0,F,employee,1,17517.812,to become educated
21516,0,-914.391429,42,bachelor's degree,0,married,0,F,business,0,51649.244,purchase of my own house
21517,0,-404.679034,42,bachelor's degree,0,civil partnership,1,F,business,0,28489.529,buying my own car
21518,0,373995.710838,59,SECONDARY EDUCATION,1,married,0,F,retiree,0,24618.344,purchase of a car
21519,1,-2351.431934,37,graduate degree,4,divorced,3,M,employee,0,18551.846,buy commercial real estate
21520,1,-4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions
21521,0,343937.404131,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,-2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property
21523,3,-3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car
21524,2,-1984.507589,40,secondary education,1,married,0,F,employee,0,13127.587,to buy a car


### Conclusion

By .info method we can see that columns 'days_employed' and 'total_income' have missing values (same amount; +2000) Missing values are quantitative. Both columns have correct data type - float64, suitable for future calculation. By printing head or tail of data, we can see that column 'days_employed' has negative rows, we need to check why. Information in 'purpose' is not standartized. Column 'education' conations different letters - lower/upper case. 

## Data preprocessing

### Processing missing values

In [20]:
df['days_employed'] = abs(df['days_employed'])

DE_mean = df['days_employed'].mean()
DE_median = df['days_employed'].median()


TI_mean = df['total_income'].mean()
TI_median = df['total_income'].median()
DE_median

2194.220566878695

In [21]:
df['days_employed'].fillna(value = DE_median,inplace = True)
df['total_income'].fillna(value = TI_mean,inplace = True)

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Conclusion

As both missing values are qualitative, we can replace them with mean or medium values.  There is no big difference in them in column 'total_income', so we can apply mean. For ''days_employed' column, firstly we have to remove negative values, as it's impossible meaning for the column. 
'Median' is equal 6 years, which is possible meaning for all ages (mostly at least). 
if more detailed - Then we still have unasseptible values - like max of ~ ~ 401 thousand working 'days_employed' (equal 183 years, which is much bigger than all options in column 'ages'.
we can convert days to ages, compare with 'ages', when it's higher - substruct childhood (~14/16), then calculate mean. 

Rows with missing values are equal for both columns, categoies in other columns - vary. Possible reason for missing values is that these numbers were taken from official sources - papers or databases, so people with empty positions just didn't provide this information.

### Data type replacement

In [22]:
df['days_employed'] = df['days_employed'].astype('int')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Conclusion

Number of days employed should be Integral, so we have to convert it.

### Processing duplicates

In [23]:
df['total_income'].value_counts()

26787.568355    2174
31791.384000       2
42413.096000       2
17312.717000       2
22435.069000       1
                ... 
27715.458000       1
23834.534000       1
26124.613000       1
28692.182000       1
41428.916000       1
Name: total_income, Length: 19349, dtype: int64

### Conclusion

We don't have any column as ID or any other personal identification, so it's impossible to identify real duplicates. From the most "personal" column, 'total_income' we see three numbers, which repeated twice (other columns are different for them, so it's not a duplicates. And huge group of 2174 positions with equal total imcome is a missing positions, which were filled with average value.


### Categorizing Data

In [26]:
df['children'] = abs(df['children'])
print(df['children'].value_counts())
print()
print(df['family_status'].value_counts())

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64


In [27]:
def status_category (row):
    status = row['family_status_id']
    children = row['children']

    if status < 2:
        if children > 0:
            return 'couple_with'
    if status < 2:
        if children <= 0:
            return 'couple_zero'
    if status > 2:
        if children > 0:
            return 'single_with'
    return 'single_zero'

In [28]:
df['status_categorization'] = df.apply(status_category, axis=1)

In [29]:
df_pivot_1 = df.pivot_table(index='status_categorization', values='debt', aggfunc='count')
df_pivot_2 = df.pivot_table(index='status_categorization', values='debt', aggfunc='sum')

print(df_pivot_1)
print()
print(df_pivot_2)

                        debt
status_categorization       
couple_with             6305
couple_zero            10252
single_with              959
single_zero             4009

                       debt
status_categorization      
couple_with             574
couple_zero             745
single_with              94
single_zero             328


In [30]:
df_pivot_new = df.pivot_table(index='children', columns='family_status', values='debt', aggfunc='sum')
df_pivot_new

family_status,civil partnership,divorced,married,unmarried,widow / widower
children,,,,,
0,229.0,55.0,516.0,210.0,53.0
1,118.0,21.0,247.0,52.0,7.0
2,30.0,7.0,145.0,9.0,3.0
3,8.0,1.0,17.0,1.0,0.0
4,0.0,0.0,3.0,1.0,0.0
5,0.0,NaN,0.0,NaN,NaN
20,3.0,1.0,3.0,1.0,0.0


### Conclusion

Column 'children' has -1 and 20 values, which shouldn't be in it. We replace -1 with 1 and just just leave 20 as they take much less than 1% of data, plus not important for catigorization.
From first 2 pivot tables we can see that there is no clear dependance on singe person or have couple or with children or not, relationship between having debt vary on 1-2 % from total.

## Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [32]:
df_pivot_family = df.pivot_table(index='children', columns = 'family_status', values='debt', aggfunc='sum')
df_pivot_family2 = df.pivot_table(index='children', columns = 'family_status', values='debt', aggfunc='count')

In [33]:
df_pivot_family

family_status,civil partnership,divorced,married,unmarried,widow / widower
children,,,,,
0,229.0,55.0,516.0,210.0,53.0
1,118.0,21.0,247.0,52.0,7.0
2,30.0,7.0,145.0,9.0,3.0
3,8.0,1.0,17.0,1.0,0.0
4,0.0,0.0,3.0,1.0,0.0
5,0.0,NaN,0.0,NaN,NaN
20,3.0,1.0,3.0,1.0,0.0


In [34]:
df_pivot_family2

family_status,civil partnership,divorced,married,unmarried,widow / widower
children,,,,,
0,2752.0,784.0,7500.0,2265.0,848.0
1,1003.0,316.0,3011.0,454.0,81.0
2,344.0,81.0,1535.0,75.0,20.0
3,56.0,11.0,249.0,8.0,6.0
4,8.0,1.0,29.0,2.0,1.0
5,2.0,NaN,7.0,NaN,NaN
20,12.0,2.0,49.0,9.0,4.0


In [35]:
pivot_fin  = df_pivot_family/df_pivot_family2
pivot_fin

family_status,civil partnership,divorced,married,unmarried,widow / widower
children,,,,,
0,0.083212,0.070153,0.068800,0.092715,0.06250
1,0.117647,0.066456,0.082033,0.114537,0.08642
2,0.087209,0.086420,0.094463,0.120000,0.15000
3,0.142857,0.090909,0.068273,0.125000,0.00000
4,0.000000,0.000000,0.103448,0.500000,0.00000
5,0.000000,NaN,0.000000,NaN,NaN
20,0.250000,0.500000,0.061224,0.111111,0.00000


In [36]:
df_pivot_ch = df.pivot_table(index='children', values='debt', aggfunc='sum')
df_pivot_ch2 = df.pivot_table(index='children', values='debt', aggfunc='count')
pivot_ch_fin  = df_pivot_ch/df_pivot_ch2

In [37]:
pivot_ch_fin

,debt
children,
0,0.075129
1,0.091470
2,0.094404
3,0.081818
4,0.097561
5,0.000000
20,0.105263


### Conclusion

There is a small correlation between having children and paying debt on time, group without children has little less chances not to pay on time in total. but even this don't work for group of devorsed separetly

- Is there a relation between marital status and repaying a loan on time?

In [38]:
df_pivot_st = df.pivot_table(index='family_status', values='debt', aggfunc='sum')
df_pivot_st2 = df.pivot_table(index='family_status', values='debt', aggfunc='count')
pivot_st_fin  = df_pivot_st/df_pivot_st2
pivot_st_fin

,debt
family_status,
civil partnership,0.092890
divorced,0.071130
married,0.075202
unmarried,0.097405
widow / widower,0.065625


### Conclusion

Widows / widowers are the most responsible category, unmarried is the least, diffirence between this two categories is more than 3%.

- Is there a relation between income level and repaying a loan on time?

In [8]:
df['total_income'].describe()

count     19351.000000
mean      26787.568355
std       16475.450632
min        3306.762000
25%       16488.504500
50%       23202.870000
75%       32549.611000
max      362496.645000
Name: total_income, dtype: float64

In [39]:
def income_cat(row):
    x = row['total_income']
    if x<=16489:
        return 'low'
    if x>16489 and x<=23203:
        return 'ave_low'
    if x>23203 and x<=32550:
        return 'ave_high'
    if x>32550:
        return 'high'

In [40]:
df['income_category'] = df.apply(income_cat, axis=1)

df_pivot_inc = df.pivot_table(index='income_category', values='debt', aggfunc='sum')
df_pivot_inc2 = df.pivot_table(index='income_category', values='debt', aggfunc='count')

pivot_inc_fin  = df_pivot_inc/df_pivot_inc2

pivot_inc_fin

,debt
income_category,
ave_high,0.085009
ave_low,0.087019
high,0.070484
low,0.079165


### Conclusion

Average (both) income categories shows higher debt rates. The lowest has the category with high income, then goes low income. Probably, bank pays more attention to these groups, as lower seemsmore risky because of low income, higher - because of bigger sums of credit.

- How do different loan purposes affect on-time repayment of the loan?

In [43]:
df_pivot_purp = df.pivot_table(index='purpose', values='debt', aggfunc='sum')
df_pivot_purp2 = df.pivot_table(index='purpose', values='debt', aggfunc='count')

pivot_purp_fin  = df_pivot_purp/df_pivot_purp2

In [41]:
pivot_purp_fin

,debt
purpose,
building a property,0.087097
building a real estate,0.076677
buy commercial real estate,0.070783
buy real estate,0.068910
buy residential real estate,0.067545
buying a second-hand car,0.075157
buying my own car,0.091089
buying property for renting out,0.079632
car,0.084848


In [42]:
df_pivot_new = df.pivot_table(index='purpose', columns='family_status', values='debt', aggfunc='count')
df_pivot_new

family_status,civil partnership,divorced,married,unmarried,widow / widower
purpose,,,,,
building a property,60.0,35.0,410.0,85.0,30.0
building a real estate,56.0,38.0,393.0,101.0,38.0
buy commercial real estate,52.0,51.0,418.0,100.0,43.0
buy real estate,55.0,38.0,426.0,75.0,30.0
buy residential real estate,51.0,40.0,397.0,97.0,22.0
buying a second-hand car,44.0,31.0,302.0,78.0,24.0
buying my own car,48.0,31.0,332.0,72.0,22.0
buying property for renting out,73.0,39.0,427.0,86.0,28.0
car,56.0,34.0,316.0,59.0,30.0


In [11]:
df.purpose.unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [12]:
car = ['car', 'second-hand', 'cars']
house = ['house', 'hous','housing', 'transactions', 'family', 'real estate',
         'commercial', 'residential', 'construction', 'property', 
         'transactions', 'commercial', 'building', 'housing', 'property', 
         'renting', 'renovation', 'real']
wedding = ['wedding', 'having a wedding', 'ceremony', 'wed']
education = ['supplementary', 'education', 'educated', 'higher', 'profile', 
             'university']

m=Mystem()


In [13]:
def lemmatization_func (line):
    lemmatized=m.lemmatize(line)
    if any(word in lemmatized for word in car):
        return 'car'
    elif any(word in lemmatized for word in house):
        return 'house'
    elif any(word in lemmatized for word in wedding):
        return 'wed'
    elif any(word in lemmatized for word in education):
        return 'edu'
    else:
        return 'house'

In [47]:
df['purpose_cat']=df['purpose'].apply(lemmatization_func)

In [48]:
df.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,status_categorization,income_category,purpose_cat
21515,1,467,28,secondary education,1,married,0,F,employee,1,17517.812,to become educated,couple_with,ave_low,edu
21516,0,914,42,bachelor's degree,0,married,0,F,business,0,51649.244,purchase of my own house,couple_zero,high,house
21517,0,404,42,bachelor's degree,0,civil partnership,1,F,business,0,28489.529,buying my own car,couple_zero,ave_high,car
21518,0,373995,59,SECONDARY EDUCATION,1,married,0,F,retiree,0,24618.344,purchase of a car,couple_zero,ave_high,car
21519,1,2351,37,graduate degree,4,divorced,3,M,employee,0,18551.846,buy commercial real estate,single_with,ave_low,house
21520,1,4529,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,couple_with,high,house
21521,0,343937,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,couple_zero,ave_high,car
21522,1,2113,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,couple_with,low,house
21523,3,3112,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,couple_with,high,car
21524,2,1984,40,secondary education,1,married,0,F,employee,0,13127.587,to buy a car,couple_with,low,car


In [49]:
df['purpose_cat'].value_counts()

house    10840
car       4315
edu       4022
wed       2348
Name: purpose_cat, dtype: int64

In [15]:
df[df['purpose_cat'] == 'other']['purpose'].value_counts()

Series([], Name: purpose, dtype: int64)

In [50]:
df_pivot_purp_cat = df.pivot_table(index='purpose_cat', values='debt', aggfunc='sum')
df_pivot_purp_cat2 = df.pivot_table(index='purpose_cat', values='debt', aggfunc='count')
pivot_purp_cat_fin  = df_pivot_purp_cat/df_pivot_purp_cat2
pivot_purp_cat_fin

,debt
purpose_cat,
car,0.093395
edu,0.091994
house,0.072140
wed,0.079216


### Conclusion

Between all purposes car and education reasons have higher debt, then other reasons like wedding or housing.
For calculations purposes divided in 4 main reasons - connected with housing, cars, wedding and education.
Dividing purposes into 4 categories shows that only 7,2%(the lowest of the house credits has problems with repaying a loan on time, while in cars category - 9,3%.

## General conclusion

Category of income status and purpose of credit has the highest impact on debt payments, comparing on other categories reviewed. 